# Introduction

This notebook uses an already trained generator to generate audio data. The first possibility is to generate a single random song and plot the waveform. The second possibility is to change the values of the latent vector individually and observe the influence on the generated song.

# Imports and Gloabl Variables

* The library numpy is used to create and load the training data and to execute mathmatical operations. 
* The library librosa is used to plot a wavefrom
* The library tensorflow is used create, save and load models and layers
* The library soundfile is used to convert wavefrom data to a WAV format
* The library pyaudio is used to play the songs in the brackground
* The library wave is used to provide the waveform data to pyaudio
* The library time is used to delay execution of code
* The library ipywidgets is used to build interactive python function to manipulate the audio data
* The library IPython is used to show the interactive sliders


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import librosa.display
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow import keras
import soundfile as sf
import pyaudio
import wave
import time
import ipywidgets as widgets
from ipywidgets import interact,interactive,fixed,interact_manual
from IPython.display import display


In [ ]:
MONITORING_DIR = "./Monitoring" # where the generator is saved to generate audio data

# Custom Layers
Same layers as defined in the notebook "model.ipynb". They are needed to load the generator

In [ ]:
class WeightedSum(keras.layers.Add):
    '''
    inherits from keras Add layer
    takes to inputs multiplies them by a factor and adds them up
    outputs the weighted sum 
    '''
    # init with default value
    def __init__(self, alpha=0.0, **kwargs):
        super(WeightedSum, self).__init__(**kwargs)   # initializes the parent class constructor
        self.alpha = keras.backend.variable(alpha, name='ws_alpha') # initializes the class variales alpha

    # output a weighted sum of inputs
    def _merge_function(self, inputs):
        # only supports a weighted sum of two inputs
        assert (len(inputs) == 2) # make sure it is only two inputs
        # ((1-a) * input1) + (a * input2)
        output = ((1.0 - self.alpha) * inputs[0]) + (self.alpha * inputs[1]) 
        return output

In [ ]:

class PixelNormalization(keras.layers.Layer):
    '''
    inherits from keras layer
    scales the feature vector for every position to unit length
    '''
    # initialize the layer
    def __init__(self, **kwargs):
        super(PixelNormalization, self).__init__(**kwargs) # initializes the parent class constructor

    # perform the operation
    def call(self, inputs):
        values = inputs**2.0 # calculate square signal values     
        mean_values = keras.backend.mean(values, axis=-1, keepdims=True) # calculate the mean signal values      
        mean_values += 1.0e-8 # ensure the mean is not zero
        l2 = keras.backend.sqrt(mean_values) # calculate the sqrt of the mean squared value (L2 norm)
        normalized = inputs / l2 # normalize values by the l2 norm
        return normalized

    # define the output shape of the layer
    def compute_output_shape(self, input_shape):
        return input_shape

In [ ]:
class Conv1DEQ_load(keras.layers.Conv1D):
    """
    inherits conv1d from keras layers 
    extends this by applying scaling of the weights with He's per-layer constant 
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs) # initialize parent constructor without weight initialization

    def build(self, input_shape):
        super().build(input_shape)
        # The number of inputs
        n = np.product([self.kernel_size[0],input_shape[-1]])
        # He initialisation constant
        self.c = np.sqrt(2/n)

    def call(self, inputs):
        
        outputs = tf.nn.conv1d(
            inputs,
            self.kernel*self.c, # scale kernel
            stride=self.strides,
            padding="SAME")

        if self.use_bias:
            outputs = tf.nn.bias_add(
                outputs,
                self.bias)

        if self.activation is not None:
            return self.activation(outputs)
        return outputs

# Generate Data

When the generator is first used to generate a song, a graph of the network is build and saved to memory. So the first time when a song is generated the execution time is large. For all further generation steps one song can be produced in under a second.

In [ ]:
latent_dim = 256

## Utils
Some funcitons needed to build the interactive interface. \
The "play_audio" funciton is taken from: https://people.csail.mit.edu/hubert/pyaudio/docs/#pyaudio.get_format_from_width \
The widgets are inspired by: https://ipywidgets.readthedocs.io/en/stable/


In [ ]:
def plot_waveform(new_song):
    '''
    plots the waveform
    new_song: waveform data as numpy array
    '''
    librosa.display.waveplot(new_song,alpha = 0.5,sr = sr)

In [ ]:

def play_audio(click):
    '''
    Plays the audio wavefrom generated by the user through the interactive sliders
    The function is executed when button b is pressed
    click: Button object
    '''
    def callback(in_data, frame_count, time_info, status):
        '''
        is called for every frame of the audio data and return the audio that is to be played
        '''
        data = wf.readframes(frame_count) # provides data frame
        return (data, pyaudio.paContinue) 
    try:
        p = pyaudio.PyAudio() # creates pyaudio object
        wf = wave.open("./generated_song.wav", 'rb') # opens the song genrated by the user
        stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                        channels=wf.getnchannels(),
                        rate=wf.getframerate(),
                        output=True,
                        stream_callback=callback) # defines the stream and calls funtion callback

        stream.start_stream() # starts the audio stream

        while stream.is_active(): # wait for stream to finish 
            time.sleep(0.1)
        stream.stop_stream() # stop stream 
        stream.close() # close stream object
        wf.close() # close audio object
        p.terminate() # terminate pyaudio object
    except: print("Please create a wavefrom first by changing slider values!")


In [ ]:
def update_plot(update):
    ''' 
    Is executed for every value change of the silders
    '''
    ax.clear() # clears plot
    new_latent_rep = [] # instantiate latent vector as a list
    for i in sliders.values(): # loop through all slider values
        new_latent_rep.append(i.value) # append slider value to latent vector
    
    new_latent_rep = np.array([new_latent_rep]) # convert to numpy array
        
    new_song = generator(new_latent_rep) # generate audio data
    new_song = new_song.numpy().reshape((number_of_samples,)) # reshape audio data
    sf.write("./generated_song.wav", new_song, sr) # save audio as wav
    target=plot_waveform(new_song) # plot waveform

## load the generator

In [ ]:
### change values
iteration = 7 # iteration from which to load the generator
latest_epoch = 4 # epoch from which to load the generator

### calculatation
sr = int(86.1328125*2**iteration)
number_of_samples= 1024*2**iteration  # calculates number of samples needed for the audio file


# load a generator model 
generator = load_model(f'./{MONITORING_DIR}/iteration_{iteration}_generator_epoch_{latest_epoch}.h5', custom_objects={
                                                                                'PixelNormalization': PixelNormalization,
                                                                                'WeightedSum': WeightedSum,
                                                                                'Conv1DEQ':Conv1DEQ_load})


## generate a single random song
In this part a single random song is generated.

### generate the waveform data from a latent vector

In [ ]:
latent_points = tf.random.uniform(shape=(1,latent_dim),minval=-1, maxval=1) # generate latent vector
X = generator.predict(latent_points) # predict waveform with the generator
sound_file = X[0].reshape((1024*2**iteration,)) # change the shape to standard wavefrom data

### plot the waveform

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1,figsize=(10,5))
plot_waveform(sound_file)

### generate the WAV soundfile

The path variables can be changed to save the song with a distinct name.

In [ ]:
path = "./random_song.wav"
sf.write(path, sound_file, sr)

## Exploring the latent space

The user is able to manipulate all variables of the latent vector that are passed to the generator. The waveform is plotted and the audio can be played.

Every slider changes one value of the latent vector. The resulting waveform is directly displayed in a new window. When the button "Play song" is pressed the shown waveform is played in the background. When the song is done playing the waveform can be changed again. At least on value has to be initally changed to display a waveform and to play a song.

With the variable "initialsize_random" the intial slider values can be set to a random value instead of all being zero.

In [ ]:
initialsize_random = False

In [ ]:
# if set to true all slider values are initialized randomly if false then all are set to zero
if initialsize_random:
    values = tf.random.uniform(shape=(1,latent_dim),minval=-1, maxval=1)[0].numpy()
else: values = tf.zeros(latent_dim).numpy()
    
# makes the plot appear in a new window
%matplotlib qt 

fig, ax = plt.subplots(1,figsize=(15,8)) # set plot size
plt.suptitle("New Song") # plot title

sliders = {} # instaiate dict for the sliders
out = widgets.Output() # widgets output object mandatory

for k in range(1,latent_dim+1): # create key value pair for the slider for every position of the latent vector
    variable = f"variable_{k}" # key name
    value = widgets.FloatSlider(min=-1,
                        max = 1,
                        value = values[k-1],
                        step=0.02,
                        description=f'{k}:') # dict value
    sliders[variable] = value # add key value pair to dict

for slider in sliders.values():
    slider.observe(update_plot, names = "value") # observe all sliders and call function update_plot when there change

controls = {} # list for the slider objects than should be displayed
for i in range(0,latent_dim+1,4): # creating a horizontal box which 4 sliders each
    name = f"c{i}"
    value = widgets.HBox(list(sliders.values())[i:i+4])

    controls[name]=value
    
        
# button design
b = widgets.Button(
    description='Play song',
    disabled=False,
    button_style='',
    tooltip='You need to change some slider values first',
    icon='' 
)    

b2 = widgets.Button(
    description='Set random values',
    disabled=False,
    button_style='',
    tooltip='Set the sliders to random values',
    icon='' 
)   
    
display(b,widgets.VBox([controls[f"c{i}"] for i in range(0,latent_dim+1,4)]+[out])) # display sliders
b.on_click(play_audio) # when button b is pressed play audio
